In [1]:
import pandas as pd
import numpy as np
import pickle, time, math
from datetime import datetime
import os 
import csv

In [2]:
data=pd.read_csv('train_data.csv')
if 'totals_transactionRevenue' in data.columns:
    y=data.pop('totals_transactionRevenue').to_frame()
x=data.copy()

C:\Users\h_agu\Desktop\machine_learning\Miniconda3\envs\tensorflow-gpu3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
x=x.drop(['visitStartTime','visitNumber','visitId','totals_hits','totals_pageviews','sessionId','fullVisitorId','date'],axis=1)

In [4]:
for col in x.columns:
    if 'visitStartTime_' in col:
        x[col]=x[col].astype(np.object)
    if 'date_' in col:
        x[col]=x[col].astype(np.object)

In [5]:
features=x.columns.tolist()
category_features=[]
category_features_idx=[]
for col in features:
    if x[col].dtypes==np.object:
        #print(col,np.object)
        category_features.append(col)
        category_features_idx.append(features.index(col))
    if x[col].dtypes==np.bool:
        #print(col,np.bool)
        category_features.append(col)
        category_features_idx.append(features.index(col))

In [6]:
from catboost import CatBoostClassifier,Pool
from sklearn.model_selection import StratifiedKFold, train_test_split

In [7]:
#xtrain,xtest,ytrain,ytest=train_test_split(x, y, test_size=0.1, random_state=42, shuffle=True, stratify=y)
_y=y.copy()

In [8]:
_y.loc[(_y['totals_transactionRevenue']>0),'totals_transactionRevenue']=1
_y.loc[(_y['totals_transactionRevenue']==0),'totals_transactionRevenue']=0

In [9]:
xtrain,xtest,ytrain,ytest=train_test_split(x, _y, test_size=0.4, random_state=42, shuffle=True, stratify=_y)

In [10]:
xtest.head()

,channelGrouping,date_day,date_dayofweek,date_dayofyear,date_month,date_quarter,date_year,device_browser,device_deviceCategory,device_isMobile,...,trafficSource_medium,trafficSource_referralPath,trafficSource_source,visitStartTime_dayofmonth,visitStartTime_dayofweek,visitStartTime_dayofyear,visitStartTime_hourofday,visitStartTime_month,visitStartTime_quarter,visitStartTime_year
440245,Social,13,3,287,10,4,2016,Safari,desktop,False,...,referral,/yt/about/,youtube.com,13,3,287,11,10,4,2016
604527,Organic Search,12,2,286,10,4,2016,Chrome,mobile,True,...,organic,?,google,13,3,287,1,10,4,2016
797156,Social,8,5,98,4,2,2017,Chrome,desktop,False,...,referral,/yt/about/pt-BR/,youtube.com,8,5,98,12,4,2,2017
661533,Organic Search,16,1,136,5,2,2017,Chrome,desktop,False,...,organic,?,google,16,1,136,15,5,2,2017
501811,Direct,22,5,203,7,3,2017,Safari,desktop,False,...,(none),?,(direct),22,5,203,11,7,3,2017


In [11]:
features=xtrain.columns.tolist()
category_features=[]
category_features_idx=[]
for col in features:
    if xtest[col].dtype == np.object or xtest[col].dtype == np.bool:
        category_features.append(col)
        category_features_idx.append(features.index(col))

In [12]:
sF=StratifiedKFold(n_splits=5,shuffle=True)
cv_split=[]
for train_idx,test_idx in sF.split(xtrain,ytrain):
    pool_train=Pool(xtrain.iloc[train_idx],ytrain.iloc[train_idx],cat_features=category_features_idx)
    pool_test=Pool(xtrain.iloc[test_idx],ytrain.iloc[test_idx],cat_features=category_features_idx)
    cv_split.append((pool_train,pool_test))

In [13]:
from hyperopt import fmin, tpe, Trials, STATUS_OK, STATUS_FAIL, hp, pyll

In [14]:
param_space = {
            'depth': hp.choice('depth', [4,6]),
            'border_count': hp.choice('border_count', [64,128]),
            'learning_rate': hp.loguniform('learning_rate', -5, 0),
            'random_strength': hp.choice('random_strength', [1,  20]),
            'one_hot_max_size': hp.choice('one_hot_max_size', [5, 25, 225]),
            'l2_leaf_reg': hp.loguniform('l2_leaf_reg', 0, np.log(10)),
            'bagging_temperature': hp.uniform('bagging_temperature', 0, 1),
            'leaf_estimation_iterations':hp.choice('leaf_estimation_iterations',[1,3,5,7,10]),
            'max_ctr_complexity':hp.quniform('max_ctr_complexity',1,5,1),
            'leaf_estimation_method':hp.choice('leaf_estimation_method',['Newton','Gradient']),
            #'class_weights': (hp.choice('non_class_weights_ratio',[1]), hp.uniform('class_weights_ratio',1,20))
        }

In [15]:
def fit(params=None, dtrain=None, dtest=None, n_estimators=None, seed=0, run_time=None, run_cv_id=0, eval_no=0, verbose=False):
    global metric,column_names
    #print(run_cv_id, eval_no)
    path="./cv_run/"+str(run_time)
    if not os.path.isdir(path):
        os.mkdir(path)
    fpath=path+"/"+str(eval_no)+"."+str(run_cv_id)
    if not os.path.isdir(fpath):
        os.mkdir(fpath)    
    params.update({"iterations": n_estimators})
    params.update({"eval_metric": metric})
    params.update({"logging_level": 'Silent'})
    #params.update({"metric_period": 100})
    params.update({"random_seed": seed})
    #params.update({"leaf_estimation_method": "Newton"})
    #params.update({"leaf_estimation_iterations" : 10})
    params.update({"rsm" : 1})
    params.update({"thread_count" : 8})
    params.update({"fold_len_multiplier": 2})
    #params.update({"max_ctr_complexity":5})
    params.update({"train_dir": fpath})
    params.update({"calc_feature_importance" : True})
    params.update({'od_type':'Iter'})
    params.update({'od_wait':30})
    
    bst = CatBoostClassifier(**params)
    bst.fit(dtrain, eval_set=dtest, use_best_model=True)
    with open(fpath + "/test_error.tsv", "r") as f:
        reader=np.array(list(csv.reader(f,delimiter='\t'))).squeeze()
    header=reader[0]
    feature=dict()
    for col, val in zip(features,bst.__dict__['_feature_importance']):
        feature.update({col:val})
    #pd.to_pickle(bst.__dict__['_feature_importance'],path+"/feature_importance."+str(eval_no)+"."+str(run_cv_id))
    pd.to_pickle(feature,path+"/feature_importance."+str(eval_no)+"."+str(run_cv_id))
    idx=(header==metric).argmax()
    results=(reader[1:reader.shape[0],idx]).astype(np.float)
    if metric=='AUC' or 'Accuracy':
        results=1-results
        
    return bst, results

In [16]:
def mean_each_iter(results):
    lengs=[]
    _=[lengs.append(len(res)) for res in results]
    mlen=np.max(lengs)
    #print(lengs,mlen)
    a=[]
    for run in results:
        a.append((np.pad(run,(0,mlen-len(run)),'constant')).tolist())
    x=np.array(a)
    #print(x.shape)
    means=[]
    for i in range(x.shape[1]):
        mean=0
        count=0
        for j in range(x.shape[0]):
            if x[j,i] > 0:
                count=count+1
                mean=mean+x[j,i]
        means.append(mean/count)
    return means

In [17]:
def run_cv(cv_pairs, params=None, n_est=None, verbose=False, run_time=None):
    global default_params,n_estimators,best_loss,hyperopt_eval_num,metric,hyperopt_evals,metric
    params = params or default_params
    n_estimators = n_est or n_estimators
    #print('run_cv',hyperopt_eval_num)
    evals_results, start_time = [], time.time()
    _loss=[]
    i=0
    for dtrain, dtest in cv_pairs:
        _, evals_result = fit(params, dtrain, dtest, n_estimators, run_time=run_time, run_cv_id=i, eval_no=hyperopt_eval_num+1)
        #evals_results.append(np.mean(evals_result,axis=0))
        evals_results.append(evals_result)
        _loss.append(np.min(evals_result))
        i=i+1
    mean_evals_results = mean_each_iter(evals_results)
    if metric=='AUC' or 'Accuracy':
        best_n_estimators = np.argmax(mean_evals_results) + 1
    else:
        best_n_estimators = np.argmin(mean_evals_results) + 1
    eval_time = time.time() - start_time
    
    
    cv_result = {'loss': mean_evals_results[best_n_estimators - 1] ,
                 'best_n_estimators': best_n_estimators, 
                 'eval_time': eval_time,
                 'status': STATUS_FAIL if np.isnan(mean_evals_results[best_n_estimators - 1]) else STATUS_OK,
                 'params': params.copy(),
                 'losses': _loss
                }
    best_loss = min(best_loss, cv_result['loss'])
    hyperopt_eval_num += 1
    cv_result.update({'hyperopt_eval_num': hyperopt_eval_num, 'best_loss': best_loss})
        
    if verbose:
        print ('[{0}/{1}]\teval_time={2:.2f} sec\tcurrent_{3}={4:.6f}\tmin_{3}={5:.6f}'.format(
                    hyperopt_eval_num, hyperopt_evals, eval_time,
                    metric, cv_result['loss'], best_loss))
    return cv_result

In [19]:
n_estimators=501
hyperopt_evals=0
metric="Logloss"
max_evals = 20
this_trials = Trials()
run_time=(datetime.now()).strftime('%Y%m%d%H%M')
hyperopt_eval_num, best_loss, split_pair_data = 0, np.inf, None
args=param_space
_ = fmin(fn=lambda args: run_cv(cv_split, params=args, n_est=n_estimators, verbose=True,run_time=run_time), 
         space=args, algo=tpe.suggest, max_evals=max_evals, trials=this_trials)

with open('./cv_run/'+run_time+'/trails.pickle','wb') as f:
    pickle.dump(this_trials,f)

[1/0]	eval_time=199.54 sec	current_Logloss=0.970020	min_Logloss=0.970020
[2/0]	eval_time=734.33 sec	current_Logloss=0.969940	min_Logloss=0.969940


KeyboardInterrupt: 

In [21]:
data.to_csv('train_data.csv',index=False)